In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Environment OK")


Environment OK


In [9]:
# Load the Dataset

df = pd.read_csv("../data/epl_final.csv")

print("Dataset shape:", df.shape)
display(df.head())
print(df.columns.tolist())


Dataset shape: (9380, 22)


,Season,MatchDate,HomeTeam,AwayTeam,FullTimeHomeGoals,FullTimeAwayGoals,FullTimeResult,HalfTimeHomeGoals,HalfTimeAwayGoals,HalfTimeResult,...,HomeShotsOnTarget,AwayShotsOnTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards
0,2000/01,2000-08-19,Charlton,Man City,4,0,H,2,0,H,...,14,4,6,6,13,12,1,2,0,0
1,2000/01,2000-08-19,Chelsea,West Ham,4,2,H,1,0,H,...,10,5,7,7,19,14,1,2,0,0
2,2000/01,2000-08-19,Coventry,Middlesbrough,1,3,A,1,1,D,...,3,9,8,4,15,21,5,3,1,0
3,2000/01,2000-08-19,Derby,Southampton,2,2,D,1,2,A,...,4,6,5,8,11,13,1,1,0,0
4,2000/01,2000-08-19,Leeds,Everton,2,0,H,2,0,H,...,8,6,6,4,21,20,1,3,0,0


['Season', 'MatchDate', 'HomeTeam', 'AwayTeam', 'FullTimeHomeGoals', 'FullTimeAwayGoals', 'FullTimeResult', 'HalfTimeHomeGoals', 'HalfTimeAwayGoals', 'HalfTimeResult', 'HomeShots', 'AwayShots', 'HomeShotsOnTarget', 'AwayShotsOnTarget', 'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls', 'HomeYellowCards', 'AwayYellowCards', 'HomeRedCards', 'AwayRedCards']


In [10]:
print(df.columns.tolist())
print(df.shape)


['Season', 'MatchDate', 'HomeTeam', 'AwayTeam', 'FullTimeHomeGoals', 'FullTimeAwayGoals', 'FullTimeResult', 'HalfTimeHomeGoals', 'HalfTimeAwayGoals', 'HalfTimeResult', 'HomeShots', 'AwayShots', 'HomeShotsOnTarget', 'AwayShotsOnTarget', 'HomeCorners', 'AwayCorners', 'HomeFouls', 'AwayFouls', 'HomeYellowCards', 'AwayYellowCards', 'HomeRedCards', 'AwayRedCards']
(9380, 22)


In [11]:
# Define the Target (Correctly)

# Target variable encoding
df["target"] = df["FullTimeResult"].map({
    "H": 2,  # Home win
    "D": 1,  # Draw
    "A": 0   # Away win
})

# Check class distribution
df["target"].value_counts(normalize=True)


target
2    0.458316
0    0.295096
1    0.246588
Name: proportion, dtype: float64

In [12]:
# Explicitly Mark Leakage Columns

leakage_cols = [
    "FullTimeHomeGoals", "FullTimeAwayGoals",
    "HalfTimeHomeGoals", "HalfTimeAwayGoals",
    "HomeShots", "AwayShots",
    "HomeShotsOnTarget", "AwayShotsOnTarget",
    "HomeCorners", "AwayCorners",
    "HomeFouls", "AwayFouls",
    "HomeYellowCards", "AwayYellowCards",
    "HomeRedCards", "AwayRedCards",
    "HalfTimeResult", "FullTimeResult"
]

print("Leakage columns (NOT usable for prediction):")
leakage_cols


Leakage columns (NOT usable for prediction):


['FullTimeHomeGoals',
 'FullTimeAwayGoals',
 'HalfTimeHomeGoals',
 'HalfTimeAwayGoals',
 'HomeShots',
 'AwayShots',
 'HomeShotsOnTarget',
 'AwayShotsOnTarget',
 'HomeCorners',
 'AwayCorners',
 'HomeFouls',
 'AwayFouls',
 'HomeYellowCards',
 'AwayYellowCards',
 'HomeRedCards',
 'AwayRedCards',
 'HalfTimeResult',
 'FullTimeResult']

In [13]:
# Keep Only Safe Columns (For Now)

base_df = df[[
    "Season",
    "MatchDate",
    "HomeTeam",
    "AwayTeam",
    "target"
]].copy()

base_df.head()


,Season,MatchDate,HomeTeam,AwayTeam,target
0,2000/01,2000-08-19,Charlton,Man City,2
1,2000/01,2000-08-19,Chelsea,West Ham,2
2,2000/01,2000-08-19,Coventry,Middlesbrough,0
3,2000/01,2000-08-19,Derby,Southampton,1
4,2000/01,2000-08-19,Leeds,Everton,2


In [14]:
# Convert Date & Sort Chronologically

# Convert date
base_df["MatchDate"] = pd.to_datetime(base_df["MatchDate"])

# Sort by time
base_df = base_df.sort_values("MatchDate").reset_index(drop=True)

base_df.head()


,Season,MatchDate,HomeTeam,AwayTeam,target
0,2000/01,2000-08-19,Charlton,Man City,2
1,2000/01,2000-08-19,Chelsea,West Ham,2
2,2000/01,2000-08-19,Coventry,Middlesbrough,0
3,2000/01,2000-08-19,Derby,Southampton,1
4,2000/01,2000-08-19,Leeds,Everton,2
